In [2]:
from htmd import *

Please cite -- HTMD: High-Throughput Molecular Dynamics for Molecular Discovery
J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. 
http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


New HTMD version (1.1.0) is available. You are currently on (1.0.26). Use 'conda update htmd' to update to the new version.


In [3]:
mol = Molecule('3PTB')
mol.filter("protein")

2016-06-29 10:41:04,922 - htmd.molecule.molecule - INFO - Attempting PDB query for 3PTB
2016-06-29 10:41:06,082 - htmd.molecule.molecule - INFO - Removed 72 atoms. 1629 atoms remaining in the molecule.


In [4]:
copied = mol.copy()
copied.filter('resname ARG LYS ASP GLU HIS CYS TYR')

2016-06-29 10:41:08,993 - htmd.molecule.molecule - INFO - Removed 1175 atoms. 454 atoms remaining in the molecule.


In [5]:
rid = np.unique(copied.get('resid')) # ia.resid also works!
rn = copied.get('resname', "name CA")

In [6]:
#He creado un monstruo... Os lo comento para que os sea mas facil de entender
#La idea es generar una pos_list (que contendra las interacciones entre aa basicos) y una neg_list (que contendra las interacciones entre aa acidos).
#He tomado el esquema que montó Mauro: 
pos_list = []
neg_list = []
for resname, resid in zip(rn, rid):
    if resname == "GLU":
        #primero, seleccionas los residuos que pueden interactuar con el residuo acido/basico de interes mediante puentes de hidrogeno
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OE1 OE2 and resid "+str(resid))
        #eliminamos resultados redundantes
        un = np.unique(a)
        for residueID in un:
            #Uso una arquitectura de try/except para curarme en salud
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                #Como he comentado en el face, hay posiciones de secuencia que contienen dos residuos, en lugar de uno.
                #Para esto, calculamos la distancia, si es mayor que 1 significa que tenemos posicion doble. En ese caso
                #itero sobre las posibles posiciones y si encuentro una que sea un acido (acido en este caso porque tratamos los acidos)
                #me la quedo y la añado al neg_list
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            #Las dos listas generadas siguen un formato de tipo: resname y resid del residuo en el que estoy
                            #y resname y resid del residuo que interactua con el primero
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    #Si no hay posiciones con multiples aa, añado residuos a la lista con normalidad
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                #En el caso de que hayan errores, por pantalla aparecera un print indicando en que posicion ocurren
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ASP":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name OD1 OD2 and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["GLU", "ASP"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            neg_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["GLU", "ASP"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        neg_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "ARG":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NH1 NH2 NE and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    print("aaaaaaaaaaa")
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue
                
    elif resname == "LYS":
        a = mol.get("resid", sel="name O N OH OG1 OG OE1 NE2 ND2 OE2 SG ND1 NH1 OD1 OD2 NZ NH2 and within 4 of name NZ and resid "+str(resid))
        #print(np.unique(a), "resid", resid)
        un = np.unique(a)
        for residueID in un:
            try:
                l = len(np.unique(mol.get("resname", sel="resid "+str(residueID))))
                if l > 1:
                    for i in range(0, l):
                        if np.unique(mol.get("resname", sel="resid "+str(residueID)))[i] in ["LYS", "ARG"] and residueID != resid:
                            res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                            tup = (resname, resid, res, residueID)
                            pos_list.append(tup)
                            print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                else:
                    
                    if np.unique(mol.get("resname", sel="resid "+str(residueID))) in ["LYS", "ARG"] and residueID != resid:
                        res = np.unique(mol.get("resname", sel="resid "+str(residueID)))
                        tup = (resname, resid, res, residueID)
                        pos_list.append(tup)
                        print(np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
            except:
                print("Error at " + np.unique(mol.get("resname", sel="resid "+str(residueID))), residueID)
                continue

['ASP'] 71
['GLU'] 80
['GLU'] 70
['GLU'] 77


In [7]:
print(pos_list)

[]


In [8]:
print(neg_list)

[('GLU', 70, array(['ASP'], dtype=object), 71), ('GLU', 70, array(['GLU'], dtype=object), 80), ('GLU', 80, array(['GLU'], dtype=object), 70), ('GLU', 80, array(['GLU'], dtype=object), 77)]


In [1]:
print(neg_list)
           

NameError: name 'neg_list' is not defined